In [31]:
import pandas as pd
prefix = '14-9F-3C-DA-5B-26'
output_path = '/Volumes/hex/ST-1871742707-res/'
p_707 = pd.read_csv(output_path + 'ST-1871742707_1.0.csv')

p_707.columns

Index(['time', 'window', 'accX', 'accX_std', 'accY', 'accY_std', 'accZ',
       'accZ_std', 'gyrX', 'gyrX_std', 'gyrY', 'gyrY_std', 'gyrZ', 'gyrZ_std',
       'heartR', 'heartR_std', 'SmoothedAirPressure', 'SAP_std',
       'UncalibratedBarometerAltitude', 'UBA_std', 'AirTemperature', 'AT_std',
       'AirPressure', 'AP_std', 'PlethysmogramGreen', 'ple_std', 'alc', 'cig',
       'mood', 'tense', 'tired', 'period', 'rumination', 'socialize',
       'socialize_val', 'sport_time', 'work_time', 'day_sleep', 'phq_1',
       'phq_2'],
      dtype='object')

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [18]:
def display_scores(score):
    print('scores:', score)
    print('Mean:', score.mean())
    print('std dev:', score.std())

In [23]:
train_set, test_set = train_test_split(p_707, test_size=0.25, random_state=42, shuffle=True)

In [24]:
X_train = train_set.loc[:,'accX':'day_sleep']
y_train_1 = train_set.loc[:, ['phq_1']].values.ravel()
y_train_2 = train_set.loc[:, ['phq_2']].values.ravel()

X_test = test_set.loc[:,'accX':'day_sleep']
y_test_1 = test_set[['phq_1']].values.ravel()
y_test_2 = test_set[['phq_2']].values.ravel()

In [25]:
X_train.head()

,accX,accX_std,accY,accY_std,accZ,accZ_std,gyrX,gyrX_std,gyrY,gyrY_std,...,mood,tense,tired,period,rumination,socialize,socialize_val,sport_time,work_time,day_sleep
18432,0.90,1.85,-1.24,2.10,9.40,2.10,-0.01,0.19,0.04,0.14,...,5,8,5,0,2,7,100,0,0,0
14663,1.88,1.62,-5.03,1.93,7.63,1.93,0.04,0.35,0.01,0.25,...,3,3,4,0,7,7,95,0,0,0
78173,-4.09,0.03,-8.48,0.03,-2.37,0.03,0.03,0.15,0.03,0.10,...,9,9,10,0,1,83,49,0,0,0
42670,1.08,0.04,-3.87,0.03,8.74,0.03,-0.03,0.20,-0.09,0.41,...,8,8,9,0,1,11,91,0,0,0
70324,1.80,0.03,-7.54,0.03,-6.14,0.03,0.05,0.20,-0.03,0.13,...,5,7,5,0,2,84,90,0,0,0


In [26]:
rnd_reg = RandomForestRegressor(n_estimators=500, max_leaf_nodes=35, n_jobs=-1 ,oob_score=True)
rnd_reg.fit(X_train, y_train_1)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=35,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [27]:
y_pred_1 = rnd_reg.predict(X_train)

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np
forest_mse = mean_squared_error(y_train_1, y_pred_1)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.4609353743747911

In [30]:
from sklearn.utils import shuffle
y_shuffled = shuffle(y_pred_1)
forest_mse_shuffled = mean_squared_error(y_train_1, y_shuffled)
forest_rmse_shuffled = np.sqrt(forest_mse_shuffled)
forest_rmse_shuffled

3.5627096587431284

In [29]:

forest_scores = cross_val_score(rnd_reg, X_train, y_train_1,scoring="neg_mean_squared_error", cv=10)

forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

scores: [0.4591379  0.4604151  0.45324259 0.45746411 0.45514638 0.45413818
 0.44515942 0.4609611  0.45722323 0.47693947]
Mean: 0.4579827466859552
std dev: 0.007655436386227206
